In [2]:
import numpy as np
import torch
from airxd_cnn.cnn import ARIXD_CNN as cmodel

In [3]:
import os
import random

In [3]:
#Test list files in directory
dir = 'data/battery1/masks'
#List all files in dir
files = os.listdir(dir)

In [8]:
np.random.seed(0)
random.sample(range(0, 11), 3)

[9, 6, 3]

In [4]:
#CNN Params and CNN itself
N = 256
M = N // 2
B = M // 4
quilter_params = {'Y': 2880, 'X': 2880,
                  'window': (N, N),
                  'step': (M, M),
                  'border': (B, B),
                  'border_weight': 0}

# TUNet params
model_params = {'image_shape': (2880, 2880),
                'in_channels': 1,
                'out_channels': 2,
                'base_channels': 8,
                'growth_rate': 2,
                'depth': 3}

# Training params
epoch = 75
batch_size = 50
lr_rate = 1e-2

model = cmodel(quilter_params, model_params, device='cuda:0', savepath = 'random')


Testing model

In [24]:
# model.load('16xmodel_11_15_2024/model_iteration_4.pt')

#Below code is for "per-epoch" saved models
state_dict = torch.load('/local/califone_backup/local_avong/AIRXD_results_1x_nopruning/3/256/repeat_4/net_epoch_50.pt')
new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
model.model.load_state_dict(new_state_dict)

model.model = model.model.to('cuda:0')

In [15]:
model.model

TUNet(
  (activation): ReLU()
  (Encode_0): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (Decode_0): Sequential(
    (0): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(8, 2, kernel_size=(1, 1), stride=(1, 1))
  )
  (Step Down 0): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=[0, 0], dilation=1, ceil_mode=False)
  (Step Up 0): ConvTranspose2d(16, 8,

In [7]:
from glob import glob

In [8]:
test_input_paths= [f for f in glob('data_modified_mask/battery5/*.tif')]
test_mask_paths = [f for f in glob('data_modified_mask/battery5/masks/*.tif')]

#Sort input/mask paths by the last part of the filename
test_input_paths.sort(key=lambda x: x.split('/')[-1].split('_')[0])
test_mask_paths.sort(key=lambda x: x.split('/')[-1].split('_')[0])

test_input_paths, test_mask_paths

(['data_modified_mask/battery5/Xin-1-00000-00001.tif',
  'data_modified_mask/battery5/Xin-1-00000-00002.tif',
  'data_modified_mask/battery5/Xin-1-00000-00003.tif',
  'data_modified_mask/battery5/Xin-1-00000-00004.tif',
  'data_modified_mask/battery5/Xin-1-00000-00005.tif',
  'data_modified_mask/battery5/Xin-1-00000-00006.tif',
  'data_modified_mask/battery5/Xin-1-00000-00007.tif',
  'data_modified_mask/battery5/Xin-1-00000-00008.tif',
  'data_modified_mask/battery5/Xin-1-00000-00009.tif',
  'data_modified_mask/battery5/Xin-1-00000-00012.tif',
  'data_modified_mask/battery5/Xin-1-00000-00013.tif',
  'data_modified_mask/battery5/Xin-1-00000-00014.tif'],
 ['data_modified_mask/battery5/masks/Xin-1-00000-00001_mask.tif',
  'data_modified_mask/battery5/masks/Xin-1-00000-00002_mask.tif',
  'data_modified_mask/battery5/masks/Xin-1-00000-00003_mask.tif',
  'data_modified_mask/battery5/masks/Xin-1-00000-00004_mask.tif',
  'data_modified_mask/battery5/masks/Xin-1-00000-00005_mask.tif',
  'data_m

In [9]:
from sklearn.metrics import confusion_matrix as CM
import imageio as iio

In [10]:
def calculate_CM(pred, labels):
    matrix = CM(labels.ravel(), pred.ravel())
    matrix_flat = matrix.ravel()
    return matrix_flat

In [25]:
results = np.zeros((4))

for i in range(len(test_input_paths)):
    im = iio.v2.volread(test_input_paths[i])
    label = iio.v2.volread(test_mask_paths[i])

    #Prediction
    pred = model.predict_old(im)

    #Do CM calc
    single_cm = calculate_CM(pred, label)

    results += single_cm



torch.Size([484, 1, 256, 256])
torch.Size([484, 1, 256, 256])
torch.Size([484, 1, 256, 256])
torch.Size([484, 1, 256, 256])
torch.Size([484, 1, 256, 256])
torch.Size([484, 1, 256, 256])
torch.Size([484, 1, 256, 256])
torch.Size([484, 1, 256, 256])
torch.Size([484, 1, 256, 256])
torch.Size([484, 1, 256, 256])
torch.Size([484, 1, 256, 256])
torch.Size([484, 1, 256, 256])


In [26]:
tn = results[0]/(results[0] + results[1]+1)
tp = results[3]/(results[3] + results[2]+1)

tn, tp

(0.9954833662388914, 0.8208049113233288)

In [ ]:
#For 1x model without pruning:
#1: 79.5
#2: 79.0
#3: 80.9
#4: 81.5
#5: 82.0

In [ ]:
#For 16x model

#1: 90.44338
#2: 80.34
#3: 86.3369
#4: 80.8867
#5: 87.52

avg = (90.44338 + 80.34 + 86.3369 + 80.8867 + 87.52) / 5
avg

85.10539600000001

# Evaluating all 5 16x models on the rest of the datasets in data_modified_mask

In [5]:
from validation import validate_model
from pathlib import Path
import pickle

In [6]:
#Evaluating the all 5 models on the rest of the datasets in modified_mask

N, d = 256, 3

n_repeats = 5
datadirs = ["battery1", "battery2", "battery3", "battery4", "Nickel"]

results = np.zeros((len(datadirs),n_repeats,4))
start = 0

#Results_master saving
results_master_path = '/local/califone_backup/local_avong/AIRXD_results_16x_final_eval_all.pkl'
if os.path.exists(results_master_path):
    with open(results_master_path, 'rb') as f:
        results_master = pickle.load(f)
else:
    results_master = {}

#Check how many we've done so far (in case it breaks)
if (N, d) in results_master:
    start = np.count_nonzero(results_master[(N, d)][0,:,0])
    results = results_master[(N, d)]

#Start validating
for repeat in range(start, n_repeats):
    print(f'Validating depth {d}, window {N} and repeat {repeat}')
    #Path
    model_path =  Path('/local/califone_backup/AIRXD-CNN/16xmodel_11_15_2024')
    model_path = model_path / f'model_iteration_{repeat}.pt'

    #Load model
    #Assume model has already been instantiated from above code
    model.load(model_path)

    #Validate
    validate_model(model, results, repeat)

    #Aggregate results
    print(results)
    tn = results[:,:,0]/(results[:,:,0] + results[:,:,1]+1)
    tp = results[:,:,3]/(results[:,:,3] + results[:,:,2]+1)

    print(f'Tn: {tn}, Tp: {tp}')

    results_master[(N, d)] = np.stack((tn, tp), axis=2)

    #Save results
    with open(results_master_path, 'wb') as f:
        pickle.dump(results_master, f)


Validating depth 3, window 256 and repeat 0
Predicting on image 0 in experiment 0
torch.Size([484, 1, 256, 256])
Predicting on image 1 in experiment 0
torch.Size([484, 1, 256, 256])
Predicting on image 2 in experiment 0
torch.Size([484, 1, 256, 256])
Predicting on image 3 in experiment 0
torch.Size([484, 1, 256, 256])
Predicting on image 4 in experiment 0
torch.Size([484, 1, 256, 256])
Predicting on image 5 in experiment 0
torch.Size([484, 1, 256, 256])
Predicting on image 6 in experiment 0
torch.Size([484, 1, 256, 256])
Predicting on image 7 in experiment 0
torch.Size([484, 1, 256, 256])
Predicting on image 8 in experiment 0
torch.Size([484, 1, 256, 256])
Predicting on image 9 in experiment 0
torch.Size([484, 1, 256, 256])
Predicting on image 10 in experiment 0
torch.Size([484, 1, 256, 256])
Predicting on image 0 in experiment 1
torch.Size([484, 1, 256, 256])
Predicting on image 1 in experiment 1
torch.Size([484, 1, 256, 256])
Predicting on image 2 in experiment 1
torch.Size([484, 1, 